In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py


In [2]:
!pip install catboost
!pip install ipywidgets
#jupyter nbextension enable --py widgetsnbextension

In [3]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

In [4]:
fill_mode = lambda col: col.fillna(col.mode()[0])
train = train.apply(fill_mode, axis=0)
train.isna().sum()

date           0
weight         0
resp_1         0
resp_2         0
resp_3         0
              ..
feature_126    0
feature_127    0
feature_128    0
feature_129    0
ts_id          0
Length: 138, dtype: int64

In [5]:
train['action'] = ((train.resp > 0) * 1).astype(int)

In [6]:
train.drop(['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4'], axis=1, inplace=True)

In [7]:
train

,date,weight,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id,action
0,86,0.859516,1,3.151305,5.467693,-0.164505,-0.189219,0.663966,0.988896,-0.358784,...,2.433699,4.282284,1.621115,4.331030,2.553220,3.799011,2.642943,3.998054,527894,0
2,86,0.590949,-1,-0.365888,0.824004,-0.293208,-0.416391,-0.599185,-0.997330,-0.358784,...,-0.702873,4.038753,-0.789767,4.133183,-1.207878,3.402796,-0.928290,3.511141,527896,0
3,86,0.172997,1,1.514607,0.596214,0.324062,0.154730,0.845069,0.521491,-0.358784,...,2.304354,1.530169,3.596848,4.613493,4.516110,3.341374,2.635798,1.535235,527897,1
6,86,1.507813,-1,-1.008999,0.242132,-0.076187,-0.073584,-0.330932,-0.634109,-0.358784,...,-1.700655,-0.376747,-0.876730,1.645537,-1.318172,1.369452,-1.352577,0.733124,527900,0
7,86,5.554003,-1,-0.391784,0.152261,-0.603803,-0.711527,-1.204166,-1.579647,-0.358784,...,-1.690686,1.656818,-1.213169,2.949877,-2.033542,2.248799,-1.838696,1.905638,527901,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1862550,499,56.694795,-1,1.538675,2.530447,2.494852,3.263345,1.613620,2.097220,-0.401539,...,-0.984942,1.129901,-1.632432,-2.169964,-2.371293,-0.889212,-1.554352,0.215984,2390444,0
1862552,499,1.650055,1,0.270380,-1.231874,-5.802676,-3.172423,-4.357278,-2.301009,1.957683,...,1.724863,-0.984278,1.413825,-1.598825,2.087731,-1.126050,1.590538,-1.250209,2390446,0
1862584,499,0.895142,-1,-0.134380,0.160580,1.292513,1.453954,0.605912,0.687598,-0.489143,...,-0.765664,-2.148415,-0.599358,-3.155134,-0.957971,-2.285314,-0.894580,-2.064227,2390478,0
1862587,499,2.967272,-1,-0.779554,-0.597258,0.674234,0.735692,-0.153732,-0.165179,-0.175335,...,1.173186,-1.215499,0.170404,-3.433334,0.496345,-2.224752,1.207851,-1.264984,2390481,0


In [8]:
y = train['action']

In [9]:
from catboost import CatBoostClassifier 

import pickle

model = CatBoostClassifier(iterations=1000, 
                           task_type="GPU",
                           devices='0:1')

model.fit(train,y, verbose=False)

pkl_filename = "/kaggle/working/pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

In [10]:
import pickle
#load our saved model 
Pkl_Filename = "./pickle_model.pkl"
with open(Pkl_Filename, 'rb') as file:  
    model = pickle.load(file)

In [11]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df.action = model.predict(test_df).to_csv('./submission.csv') #make your 0/1 prediction here
    env.predict(sample_prediction_df)

CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:81: At position 2 should be feature with name feature_0 (found feature_1).